In [1]:
!pip install pandas sqlalchemy pymysql
import mysql.connector

In [2]:
import pandas as pd

csv_path = '/Users/jinkyukim/Documents/IU MS Data Science/D532 - Applied Database Technologies/Formatted_Schedule_Slots__all_rows_.csv'  # adjust if needed
df = pd.read_csv(csv_path)
df.head()

,initials,role,week_of_month,day_of_week,session,location_odd,is_operating,location_even,details
0,RR,S,1,1,FD,St. Vincent Clinic,0,NaN,NaN
1,MK,S,1,1,FD,Riley OR,1,NaN,NaN
2,RM,S,1,1,FD,Riley OR,1,NaN,NaN
3,BW,S,1,1,FD,ROC OR,1,NaN,NaN
4,PD,S,1,1,FD,ESK OR,1,NaN,NaN


In [3]:
# Replace all NaN (missing) values with None
df = df.where(pd.notnull(df), None)

In [4]:
# arguments to pass it as a config to a connection object.
config = {
  'user': 'root',
  'password': 'TKDrlawlsrb15915',
  'host': 'localhost',
  'port': 3306,
  'database': 'Final_Project',
  'raise_on_warnings': True
}

In [5]:
# The below method establishes the session with MySQL server 
mydb = mysql.connector.connect(**config)

# Cursor object executes the SQL statements and interact with the MySQL server using a MySQLConnection object
cursor = mydb.cursor(dictionary=True)

In [6]:
cursor.execute("""
DROP TABLE IF EXISTS staging_templates;
""")

mydb.commit()

In [7]:
# Create the staging table 
cursor.execute("""
CREATE TABLE IF NOT EXISTS staging_templates (
  initials        VARCHAR(4),
  role            CHAR(2),
  week_of_month   INT,
  day_of_week     INT,
  session         ENUM('AM','PM','FD'),
  location_odd    VARCHAR(100),
  is_operating    INT,
  location_even   VARCHAR(100),
  details         TEXT
);
""")

mydb.commit()

In [8]:
insert_stmt = """
INSERT INTO staging_templates
  (initials, role, week_of_month, day_of_week, session, location_odd, is_operating, location_even, details)
VALUES
  (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

for _, row in df.iterrows():
    cursor.execute(insert_stmt, tuple(row))
mydb.commit()

In [9]:
#Insert into providers table
cursor.execute("""
INSERT IGNORE INTO providers (initials, role)
SELECT DISTINCT initials, role FROM staging_templates
WHERE initials IS NOT NULL AND role IS NOT NULL;
""")
mydb.commit()

In [10]:
#Insert into activity types table
cursor.execute("""
INSERT IGNORE INTO activity_types (activity_name, is_operating)
SELECT DISTINCT
        CASE
            WHEN is_operating = 1 THEN 'OR'
            WHEN location_odd LIKE '%Clinic%' THEN 'Clinic'
            WHEN location_odd LIKE '%Admin%' THEN 'Admin'
            WHEN location_odd LIKE '%Research%' THEN 'Research'
            WHEN TRIM(location_odd) = 'OFF' THEN 'Off'
            ELSE 'Other'
        END AS activity_name,
        is_operating
FROM staging_templates;
""")

In [11]:
#Insert into location (odd month)
cursor.execute("""
INSERT IGNORE INTO locations (location_code, location_name, activity_type_id)
SELECT
    TRIM(location_odd) AS location_code,
    TRIM(location_odd) AS location_name,
    at.activity_type_id
FROM staging_templates AS st
JOIN activity_types AS at
  ON at.activity_name = (
      CASE
         WHEN st.is_operating = 1 THEN 'OR'
         WHEN st.location_odd LIKE '%Clinic%' THEN 'Clinic'
         WHEN st.location_odd LIKE '%Admin%' THEN 'Admin'
         WHEN st.location_odd LIKE '%Research%' THEN 'Research'
         WHEN TRIM(st.location_odd) = 'OFF' THEN 'Off'
         ELSE 'Other'
       END
    )
WHERE st.location_odd IS NOT NULL;
""")
mydb.commit()

In [12]:
# Insert locations from location (even month)
cursor.execute("""
INSERT IGNORE INTO locations (location_code, location_name, activity_type_id)
SELECT
    TRIM(location_even) AS location_code,
    TRIM(location_even) AS location_name,
    at.activity_type_id
FROM staging_templates AS st
JOIN activity_types AS at
  ON at.activity_name = (
      CASE
         WHEN st.is_operating = 1 THEN 'OR'
         WHEN st.location_even LIKE '%Clinic%' THEN 'Clinic'
         WHEN st.location_even LIKE '%Admin%' THEN 'Admin'
         WHEN st.location_even LIKE '%Research%' THEN 'Research'
         WHEN TRIM(st.location_even) = 'OFF' THEN 'Off'
         ELSE 'Other'
       END
    )
WHERE st.location_even IS NOT NULL;
""")
mydb.commit()

In [13]:
# Insert into schedule_templates
cursor.execute("""
INSERT IGNORE INTO schedule_templates
        (provider_id,
         week_of_month,
         day_of_week,
         session,
         location_id_odd,
         location_id_even,
         details)
SELECT
    p.provider_id,                 -- provider
    st.week_of_month,
    st.day_of_week,
    st.session,

    /* always use odd-month room */
    MIN(lo.location_id)                    AS location_id_odd,

    /* fill even-month room only if provided, else NULL */
    MIN(le.location_id)                    AS location_id_even,

    MAX(st.details)                        AS details
FROM   staging_templates      st
JOIN   providers              p  ON p.initials      = st.initials
JOIN   locations              lo ON lo.location_code = TRIM(st.location_odd)
LEFT   JOIN locations         le ON le.location_code = TRIM(st.location_even)

GROUP BY
    p.provider_id,
    st.week_of_month,
    st.day_of_week,
    st.session;
""")
mydb.commit()

In [14]:
import mysql.connector
import pandas as pd
from datetime import datetime, timedelta

def generate_dates(start_date, end_date):
    return pd.date_range(start=start_date, end=end_date, freq='D')

start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 12, 31)
dates_df = pd.DataFrame({'schedule_date': generate_dates(start_date, end_date)})

# Calculate week_of_month and day_of_week
def week_of_month(dt):
    first_day = dt.replace(day=1)
    dom = dt.day
    adjusted_dom = dom + first_day.weekday()
    return int((adjusted_dom - 1) / 7) + 1

dates_df['day_of_week'] = dates_df['schedule_date'].dt.weekday + 1   # Monday=1, Sunday=7
dates_df['week_of_month'] = dates_df['schedule_date'].apply(week_of_month)
# Only weekdays, Mon-Fri
dates_df = dates_df[dates_df['day_of_week'].between(1, 5)].reset_index(drop=True)

# Fetch templates
templates_query = """
SELECT template_id, provider_id, week_of_month, day_of_week, session,
       location_id_odd, location_id_even, details
FROM schedule_templates
"""
templates_df = pd.read_sql(templates_query, mydb)

# Join dates to templates 
instances_df = pd.merge(
    dates_df,
    templates_df,
    on=['week_of_month', 'day_of_week'],
    how='inner'
)

# Decide location_id per row (odd/even logic) 
def select_location_id(row):
    # Even week and location_id_even not null: use location_id_even
    if row['week_of_month'] % 2 == 0 and pd.notnull(row['location_id_even']):
        return int(row['location_id_even'])
    # Else, use location_id_odd
    return int(row['location_id_odd'])

instances_df['location_id'] = instances_df.apply(select_location_id, axis=1)

# Prepare for insert 
instances_df['generated_from_template'] = 1
instances_df['notes'] = instances_df['details']

# Only keep needed columns in the correct order
to_insert = instances_df[[
    'provider_id', 'schedule_date', 'session', 'location_id',
    'template_id', 'generated_from_template', 'notes'
]]

print(to_insert.head())  # Preview

# Insert into schedule_instances 
insert_stmt = """
INSERT IGNORE INTO schedule_instances
(provider_id, schedule_date, session, location_id, template_id, generated_from_template, notes)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

for _, row in to_insert.iterrows():
    cursor.execute(
        insert_stmt,
        (
            row['provider_id'],
            row['schedule_date'].date(),  # as date
            row['session'],
            row['location_id'],
            row['template_id'],
            row['generated_from_template'],
            row['notes']
        )
    )
mydb.commit()

print(f"Inserted {len(to_insert)} schedule_instances.")

cursor.close()
mydb.close()

/var/folders/y9/2lv0xbjn4mxbrhk8_c_481rm0000gn/T/ipykernel_76236/2161020935.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  templates_df = pd.read_sql(templates_query, mydb)


   provider_id schedule_date session  location_id  template_id  \
0            4    2025-01-01      FD            2           27   
1            2    2025-01-01      FD            2           28   
2            6    2025-01-01      FD            4           71   
3            8    2025-01-01      FD            6          131   
4            5    2025-01-01      FD            6          132   

   generated_from_template notes  
0                        1  None  
1                        1  None  
2                        1  None  
3                        1  None  
4                        1  None  
Inserted 3993 schedule_instances.
